In [ ]:
import sys
import os

# 添加 src 到路径，便于导入
sys.path.append(os.path.abspath(os.path.join(os.path.dirname("__file__"), "../src")))
print("当前工作目录:", os.getcwd())

In [ ]:
from CADAlgo.parser import *
import display_class

In [ ]:
with open("input.json", "r", encoding="utf-8") as f:
    data = json.load(f)

schema = parse_schema(data)

# to 

_ = False

In [ ]:
display_class.plot_schema(schema)